In [2]:
data = [
    {"user_id": 1, "item_id": 1, "rating": 4},
    {"user_id": 1, "item_id": 2, "rating": 5},
    {"user_id": 1, "item_id": 3, "rating": 3},
    {"user_id": 2, "item_id": 1, "rating": 4},
    {"user_id": 2, "item_id": 2, "rating": 3},
    {"user_id": 2, "item_id": 4, "rating": 5},
    {"user_id": 3, "item_id": 1, "rating": 5},
    {"user_id": 3, "item_id": 3, "rating": 2},
    {"user_id": 3, "item_id": 4, "rating": 3},
    {"user_id": 4, "item_id": 2, "rating": 5},
    {"user_id": 4, "item_id": 3, "rating": 4},
    {"user_id": 4, "item_id": 4, "rating": 4}
]

## Langkah 1: mengambil data
Pertama, kita akan mengambil data darai variabel data dan mengonversinya menjadi pandas DataFrame.


In [14]:
import pandas as pd


df = pd.DataFrame(data)
df

,user_id,item_id,rating
0,1,1,4
1,1,2,5
2,1,3,3
3,2,1,4
4,2,2,3
5,2,4,5
6,3,1,5
7,3,3,2
8,3,4,3
9,4,2,5


## Langkah 2: Membangun Matriks Utility


In [15]:
# Membuat matriks pivot untuk rating
utility_matrix = df.pivot_table(values='rating', index='user_id', columns='item_id', fill_value=0)
utility_matrix

item_id,1,2,3,4
user_id,,,,
1,4.0,5.0,3.0,0.0
2,4.0,3.0,0.0,5.0
3,5.0,0.0,2.0,3.0
4,0.0,5.0,4.0,4.0


## Langkah 3: Menghitung Similarity Antar Pengguna
Menggunakan cosine similarity untuk menghitung kemiripan antar pengguna:

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

# Menghitung cosine similarity antar pengguna
user_similarity = cosine_similarity(utility_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=utility_matrix.index, columns=utility_matrix.index)
user_similarity_df

user_id,1,2,3,4
user_id,,,,
1,1.000000,0.620000,0.596481,0.693074
2,0.620000,1.000000,0.802955,0.655610
3,0.596481,0.802955,1.000000,0.429735
4,0.693074,0.655610,0.429735,1.000000


## Langkah 4: Membuat Fungsi Rekomendasi
Mencari pengguna serupa dan memberikan rekomendasi item:

In [17]:
def get_user_recommendations(user_id, utility_matrix, user_similarity_df, num_recommendations=2):
    # Mendapatkan pengguna serupa
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index.tolist()
    
    # Mengambil item yang sudah dirating oleh user yang ingin direkomendasikan
    user_ratings = utility_matrix.loc[user_id]
    rated_items = user_ratings[user_ratings > 0].index.tolist()
    
    # Menyimpan skor untuk setiap item yang akan direkomendasikan
    scores = {}
    for similar_user in similar_users:
        if similar_user == user_id:
            continue
        similar_user_ratings = utility_matrix.loc[similar_user]
        for item, rating in similar_user_ratings.items():
            if item not in rated_items:
                if item not in scores:
                    scores[item] = user_similarity_df[user_id][similar_user] * rating
                else:
                    scores[item] += user_similarity_df[user_id][similar_user] * rating
    
    # Mengurutkan skor dan memilih item teratas
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    recommendations = sorted_scores[:num_recommendations]
    return recommendations

# Contoh mendapatkan rekomendasi untuk user_id=1
recommendations = get_user_recommendations(1, utility_matrix, user_similarity_df)
for item, score in recommendations:
    print(f"Item ID: {item}, Score: {score}")


Item ID: 4, Score: 7.661736726472196
